# Запуск Модульных Тестов (Unit Tests)

Этот ноутбук предназначен для запуска комплекта модульных тестов (unit tests) проекта и отображения результатов. Тесты написаны с использованием стандартной библиотеки Python `unittest`.

**Перед запуском тестов:**
- Убедитесь, что все необходимые зависимости проекта установлены.
- Крайне важно, чтобы демонстрационные данные для тестов были сгенерированы. Если вы этого еще не сделали, запустите скрипт `tests/test_data_setup.py` из корневой директории проекта. Многие тесты полагаются на наличие этих данных.

Тесты проверяют корректность работы отдельных классов и их методов.

In [ ]:
import unittest
import os
import sys

# --- Настройка sys.path для корректного обнаружения тестов и модулей проекта ---
# Используем текущую рабочую директорию и поднимаемся на уровень выше, если ноутбук в notebooks/
current_dir = os.getcwd()
project_root = current_dir
if os.path.basename(current_dir) == "notebooks":
    project_root = os.path.abspath(os.path.join(current_dir, '..'))

if project_root not in sys.path:
    sys.path.insert(0, project_root)
print(f"Project root (добавлен в sys.path): {project_root}")

# Директория, где находятся файлы с тестами
# Предполагается, что это папка 'tests' в корне проекта
test_dir = os.path.join(project_root, 'tests')
print(f"Директория с тестами: {test_dir}")

# --- Обнаружение тестов ---
# Загрузчик тестов будет искать файлы, соответствующие шаблону test*.py в test_dir
loader = unittest.TestLoader()
suite = None # Инициализируем suite
try:
    if not os.path.isdir(test_dir):
        print(f"ОШИБКА: Директория с тестами не найдена: {test_dir}")
        print("Пожалуйста, проверьте структуру проекта.")
    else:
        suite = loader.discover(start_dir=test_dir, pattern='test_*.py')
        if suite and suite.countTestCases() > 0:
            print(f"Тесты успешно обнаружены. Общее количество тест-кейсов: {suite.countTestCases()}")
        elif suite:
            print("Тесты обнаружены, но не найдено ни одного тест-кейса (возможно, пустые файлы тестов или неправильный pattern).")
        else:
            print("Не удалось обнаружить тесты. loader.discover вернул None.")
except ImportError as e:
    print(f"Ошибка импорта при обнаружении тестов: {e}")
    print("Убедитесь, что все зависимости установлены и sys.path настроен корректно.")
    print(f"Текущий sys.path: {sys.path}")
except Exception as e:
    print(f"Произошла ошибка при обнаружении тестов: {e}")

## Запуск Тестов и Отображение Результатов

Ячейка ниже выполнит все обнаруженные тесты. Результаты будут выведены непосредственно под ячейкой.
- **`.`** (точка) означает успешно пройденный тест.
- **`F`** означает тест, который провалился (assertion failure).
- **`E`** означает ошибку во время выполнения теста (неожиданное исключение).

In [ ]:
if suite is not None and suite.countTestCases() > 0:
    print(f"\nЗапуск {suite.countTestCases()} тестов...\n")
    # Средство выполнения тестов с выводом в текстовом формате
    # verbosity=2 для более детального вывода (имена тестов)
    # Используем buffer=True для перехвата вывода от самих тестов (например, print)
    # и вывода его только для проваленных тестов.
    runner = unittest.TextTestRunner(verbosity=2, stream=sys.stdout, buffer=True)
    results = runner.run(suite)

    print("\n--- Итоги тестирования ---")
    print(f"Всего тестов запущено: {results.testsRun}")
    passed_count = results.testsRun - len(results.errors) - len(results.failures)
    print(f"Успешно пройдено: {passed_count}")
    print(f"Провалено (Failures): {len(results.failures)}")
    if results.failures:
        print("  Детали проваленных тестов (Failures):")
        for test, traceback_text in results.failures:
            print(f"    Тест: {test}")
            # Раскомментируйте следующую строку, чтобы увидеть полный трейсбек для каждого проваленного теста
            # print(traceback_text)
    print(f"Ошибки выполнения (Errors): {len(results.errors)}")
    if results.errors:
        print("  Детали ошибок выполнения (Errors):")
        for test, traceback_text in results.errors:
            print(f"    Тест: {test}")
            # Раскомментируйте следующую строку, чтобы увидеть полный трейсбек для каждой ошибки
            # print(traceback_text)
    if hasattr(results, 'skipped') and results.skipped:
        print(f"Пропущено (Skipped): {len(results.skipped)}")
        # Можно добавить детализацию для пропущенных тестов, если необходимо
        # for test, reason in results.skipped:
        #     print(f"    Тест: {test}, Причина: {reason}")
elif suite is not None and suite.countTestCases() == 0:
    print("Тесты были обнаружены, но не найдено ни одного тест-кейса для запуска.")
else:
    print("Тесты не были обнаружены или произошла ошибка при их загрузке. Запуск невозможен.")